In [ ]:
%run dataImport.ipynb
%run Plots.ipynb
%run HelperFunctions.ipynb

In [ ]:
data = merge_data()
screw = data[0]
tile = data[1]
capsule = data[2]

# MVTEC

In [ ]:
def create_or_append_to_txt_file(file_path, time):
    '''
    function to append time for the autogluon
    '''
    try:
        with open(file_path, "a") as file:
            file.write(str(time)+"\n")
    except FileNotFoundError:
        with open(file_path, "w") as file:
            file.write(str(time)+"\n")

def create_or_append_to_csv_file(file_path, data, count_all):
    '''
    function to append metrics to a csv file
    '''
    try:
        # try to load the file
        df = pd.read_csv(file_path)
        # save the metrics
        df.loc[len(df)] = data
        df.to_csv(file_path, index=False)
    except:
        
        # if file does not exists, make file
        df_metrics_keras = pd.DataFrame(columns=["Accuracy", "ROC AUC", "Precision", "F1-score", "Time"])
        df_metrics_keras.loc[len(df_metrics_keras)] = data
        df_metrics_keras.to_csv(file_path, index=False)

# AutoKeras

In [ ]:
import os
import autokeras as ak
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

def classify(data, directory_name, time_name, metric_name):
    seeds = [1,2,3,4,5,6,7,8,9,10]
    time_autogluon = []
    count_all = 0
    
    
    count = 0
    
    for j in seeds:
        # load dataset
        train_keras, test_keras = prepare_data(data,j)
        X_test_keras, Y_test_keras = to_array(test_keras)
        X_train_keras, Y_train_keras = to_array(train_keras)
    
        print(X_test_keras.shape)
        print(Y_test_keras.shape)
        print(X_train_keras.shape)
        print(X_train_keras.shape)
        
        #directory_name = str(0)
        if not os.path.exists(directory_name):
            os.makedirs(directory_name)
        # set the classifier
        clf = ak.ImageClassifier(overwrite=True, 
                                 seed = j,
                                 project_name="plots_im/Keras/" + directory_name + "/"+ str(j), 
                                 max_trials = 50,
                                 metrics = ['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()] )
               
        from datetime import datetime as dt
        start_time = dt.now()
    
        # fit the model
        hist = clf.fit(X_train_keras, Y_train_keras, epochs = 50)
        
        end_time = dt.now()   
        # save the building time to give autogluon the same amount of time
        elapsed_seconds = (end_time - start_time).total_seconds()
        time_autogluon.append(elapsed_seconds)
        elapsed_min = divmod(elapsed_seconds, 60)
        # print the elapsed time
        time_passed = f"{int(elapsed_min[0])}m{int(elapsed_min[1])}s"
        print("Total fitting time: ", f"{int(elapsed_min[0])}m{int(elapsed_min[1])}s")
    
        # print model summarys
        print(clf.tuner.results_summary())
        
        eval = clf.evaluate(X_test_keras, Y_test_keras)
        print(eval)
    
        if (eval[3] + eval[4]) !=0:
            f1_score = 2 * (eval[3] * eval[4]) / (eval[3] + eval[4])
        else:
            f1_score = 1
        # add to dataFrame
        row_f = {"Accuracy": eval[1], 'ROC AUC': eval[2], 'Precision': eval[3], "F1-score":f1_score,"Time":time_passed}
        # save the metrics
        #df_metrics_keras.loc[count_all] = row_f
    
    
        create_or_append_to_txt_file(time_name,elapsed_seconds)
        create_or_append_to_csv_file(metric_name, row_f, count_all)
        
        # get confusion matrix
        y_prediction_keras = clf.predict(X_test_keras, batch_size = 1) 
        print('The confusion matrix is: ') 
        print(str(confusion_matrix(Y_test_keras, y_prediction_keras)))
        
        cm = confusion_matrix(Y_test_keras, y_prediction_keras)
        print(cm)
        
        plt.figure(figsize=(6, 4))
        sns.heatmap(cm, annot=True, fmt='d', cmap="Greens", cbar=True,
                xticklabels=['0', '1'], 
                yticklabels=['0', '1'])
        plt.xlabel('Predicted label')
        plt.ylabel('True label')
        plt.savefig("plots_im/Keras/Keras" + directory_name +  str(j))# + name)
        plt.show()
        
        clf.tuner.results_summary()
        model = clf.export_model()
    
        # Save the model summary to a text file
        str_name = "plots_im/Keras/" + directory_name +".txt"
        model_summary_str = ""
        model.summary(print_fn=lambda x: print(x, end="\n", file=open(str_name, "a")))
    
       
        count += 1
        count_all += 1



In [ ]:
classify(screw,"0", "MVTEC_screw.txt", "plots_im/Keras/Keras_metrics_screw.csv")

In [ ]:
classify(tile,"1", "MVTEC_tile.txt", "plots_im/Keras/Keras_metrics_tile.csv")
        

In [ ]:
classify(capsule,"2", "MVTEC_capsule.txt", "plots_im/Keras/Keras_metrics_capsule.csv")